## Google Geocoding

* Now that everyone has an API Key (Congratulations!), it is time to start using it!

* You can utilize the Google Maps Geocoding API to turn addresses into latitudinal and longitudinal coordinates.

  * This process of converting an address to coordinates is called **geocoding**.

  * Since many APIs only understand locations formatted in terms of latitude/longitude, geocoding will be very valuable in translating addresses into data that APIs - like the Google Places API - can understand.

    * Google's API is not free and if credit card information is provided, Google will charge past a certain usage point. **It is a good idea to avoid pushing your API key to github by using adding the `config.py` to their `.gitignore` file or using environment variables.**

In [ ]:
# Dependencies
import requests
import json
import pandas as pd

from pprint import pprint

# Google developer API key
import sys
sys.path.append('../..')
from config import gkey
from config import owm_api


file = pd.read_csv("winedata.csv")

file.head()

In [ ]:
file["winery name"] = file["winery"] + " Winery"
file.head()

In [ ]:
#For test purposes, just looking at one country
just_brazil = file.loc[file["country"]=="Brazil"]

In [ ]:
# Target location

winery_response =[]

# Run API calls for each winery in list
for winery in just_brazil["winery name"]:

    target_location = winery + " " + just_brazil["country"]
    
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_location, gkey)
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
   
    # Add response to our list
    winery_response.append(geo_data)        

In [ ]:
# Collect and store latitude and longitude for each winery
#winery_list = []
lat = []
lng = []

i=0

# Loop through wineries and append coordinate lists
for w in winery_response:
    try:
        latitude = winery_response[i]["results"][0]["geometry"]["location"]["lat"]
        longitude = winery_response[i]["results"][0]["geometry"]["location"]["lng"]
        
  #      winery_list.append(w)
        lat.append(latitude)
        lng.append(longitude)
        
        i = i+1
    except:
        lat.append("lat error")
        lng.append("lng error")
    
        i = i+1

In [ ]:
# Add coordinates to dataframe
just_brazil["latitude"] = lat
just_brazil["longitude"] = lng

In [ ]:
# Drop rows without location data
#just_brazil = just_brazil.loc[just_brazil["latitude"] != "na"]
just_brazil

In [ ]:
import gmaps
import numpy as np


# Configure gmaps with API key
gmaps.configure(api_key=gkey)


# Store 'Lat' and 'Lng' into  locations 
locations = just_brazil[["latitude", "longitude"]].astype(float)

In [ ]:
# Create winery symbol layer
winery_layer = gmaps.marker_layer(
    locations
)


fig = gmaps.figure()
fig.add_layer(winery_layer)

fig

In [ ]:
from config import weather_api

weather_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?q=" 

In [ ]:
type(just_brazil[["longitude", "latitude"]])
brazil_coords = just_brazil[["longitude", "latitude"]]
type(brazil_coords.iloc[1,1])

In [ ]:
latitude_float = [float(x) for x in brazil_coords["latitude"]]

brazil_coords["latitude"] = latitude_float

type(brazil_coords.iloc[1,1])

In [ ]:
# Northern hemisphere growing season weather lists
jul = []
aug = []
sep = []

# Southern hemisphere growing season weather lists
jan = []
feb = []
mar = []

# Loop through wineries and append weather lists


for index, row in brazil_coords.iterrows():
#    try:
        if row["latitude"] < 0:
            latit = (row["latitude"])
            longi = (row["longitude"])
        
 # Run a request to endpoint and convert result to json
            jul_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-07-01&enddate=2011-07-31&tp=24&format=json").json()
            aug_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-08-01&enddate=2011-08-31&tp=24&format=json").json()
            sep_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-09-01&enddate=2011-09-30&tp=24&format=json").json()
                
            jul.append(jul_weather)
            aug.append(aug_weather)
            sep.append(sep_weather)
            
            print("working")
        
#    except:
#        jul.append("error")
#        aug.append("error")
#        sep.append("error")
        
        print("not working")
        else:
#            try:
                latit = (row["latitude"])
                longi = (row["longitude"])

                jan_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-01-01&enddate=2011-01-31&tp=24&format=json").json()
                feb_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-02-01&enddate=2011-02-28&tp=24&format=json").json()
                mar_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-03-01&enddate=2011-03-31&tp=24&format=json").json()

                jan.append(jul_weather)
                feb.append(aug_weather)
                mar.append(sep_weather)

                print("working")
 #           except:
 #               jan.append("error")
  #              feb.append("error")
   #             mar.append("error")   

    #            print("not working")

                    
       